<center>

# **PREDICTIVE METHOD**<br>
# **RANDOM FOREST CLASSIFIER - WITH PCA**<br>

by: Ly Nguyen

</center>


In [1]:
# Import necessary libraries for this notebook: 

# Read from SQLite database and load to a pandas dataframe
import os
import sqlite3
import pandas as pd

# For using arrays 
import numpy as np

# For ML work (data preprocessing, hyperparameter tuning, Random Forest Classifier, training & testing sets, and stratified sampling)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder 
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.utils.class_weight import compute_class_weight


# For Dimensionality Reduction
from sklearn.decomposition import PCA
import umap.umap_ as umap
from sklearn.preprocessing import StandardScaler

# For model evaluation, including explainability:  
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, balanced_accuracy_score, make_scorer
from sklearn.utils.class_weight import compute_class_weight
import statsmodels.api as sm
import shap

# For data visualization 
import matplotlib.pyplot as plt
import seaborn as sns

# For saving the model into a pkl file
import joblib



c:\ProgramData\miniconda3\envs\DSI_participant\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the saved df_prelim parquet file: 
relative_path = os.path.join("..", "src", "df_reduced.parquet")
df_reduced = pd.read_parquet(relative_path)

In [3]:
# Define X and y:
X = df_reduced.drop(columns=['delayType'])  # Use parentheses with the 'columns' argument
y = df_reduced['delayType']


### *STANDARD SCALE THE FEATURES*

In [4]:

# Standardize the features for UMAP & PCA 
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### *PCA*

In [5]:
# Ensure PCA is performed with enough components
pca = PCA(n_components=35)   
X_pca = pca.fit_transform(X_scaled)

# Recalculate cumulative explained variance
cumulative_variance = pca.explained_variance_ratio_.cumsum()
print("Cumulative Variance:", cumulative_variance)

# Find the number of components needed for 80% variance
components_for_80 = (cumulative_variance >= 0.80).argmax() + 1
print(f"Number of components needed for 80% variance: {components_for_80}")


Cumulative Variance: [0.04079766 0.07287864 0.10322336 0.13222366 0.16013663 0.18609475
 0.21149868 0.23675769 0.26160917 0.28581807 0.30985806 0.33361644
 0.35733253 0.3805795  0.40368882 0.42659086 0.44931946 0.47174748
 0.49384462 0.51578636 0.53739628 0.55866663 0.57979573 0.60065216
 0.62117728 0.64135552 0.66144925 0.68145713 0.70140597 0.72123061
 0.74100842 0.76063965 0.78022816 0.79969704 0.81868354]
Number of components needed for 80% variance: 35


In [6]:
# Perform PCA with [number] components
pca = PCA(n_components=35)
X_pca = pca.fit_transform(X_scaled)

# Convert to DataFrame
X_pca_df = pd.DataFrame(X_pca, columns=[f'PC{i+1}' for i in range(35)])

# Save the transformed dataset
X_pca_df.to_csv('pca_transformed_data_top_components.csv', index=False)

# Print final variance
explained_variance_ratio = pca.explained_variance_ratio_
print("Explained Variance Ratio for top components:", explained_variance_ratio)
print("Total Variance Captured by top components:", sum(explained_variance_ratio))


Explained Variance Ratio for top components: [0.04079766 0.03208098 0.03034472 0.0290003  0.02791297 0.02595812
 0.02540393 0.02525901 0.02485148 0.0242089  0.02403999 0.02375837
 0.02371609 0.02324697 0.02310933 0.02290204 0.0227286  0.02242803
 0.02209714 0.02194174 0.02160992 0.02127036 0.02112909 0.02085643
 0.02052512 0.02017824 0.02009373 0.02000788 0.01994884 0.01982464
 0.0197778  0.01963123 0.01958852 0.01946887 0.0189865 ]
Total Variance Captured by top components: 0.818683535789138


In [7]:
# Stratified sampling to split the PCA-transformed data
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(
    X_pca_df, y, test_size=0.3, random_state=42, stratify=y
)

In [8]:
# Save the PCA transformed training & test sets
X_train_pca.to_csv('X_train_pca.csv', index=False)
X_test_pca.to_csv('X_test_pca.csv', index=False)
y_train_pca.to_csv('y_train_pca.csv', index=False)
y_test_pca.to_csv('y_test_pca.csv', index=False)


In [21]:


# Apply balanced class weights on the training data
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train_pca),
    y=y_train_pca
)
class_weight_dict = dict(zip(np.unique(y_train_pca), class_weights))

# Define a Random Forest Classifier
rf = RandomForestClassifier(random_state=42, class_weight=class_weight_dict)

# Define hyperparameter grid for Random Forest tuning
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    }

# Apply stratified sampling on cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define a scorer for balanced accuracy
scorer = make_scorer(balanced_accuracy_score)

# GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring=scorer,
    cv=cv,
    n_jobs=-1,
    verbose=2
)

# Fit the model on training data
grid_search.fit(X_train_pca, y_train_pca)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)


Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best Hyperparameters: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 150}


In [22]:
# Get the best estimator after GridSearchCV
best_rf_pca = grid_search.best_estimator_

In [23]:
# Evaluate the model on the test set
y_pred_pca = best_rf_pca.predict(X_test_pca)



In [24]:
# Generate the classification report
class_report_pca = classification_report(y_test_pca, y_pred_pca)
print("Classification Report:\n", class_report_pca)

# Calculate and print the balanced accuracy score
balanced_acc_pca = balanced_accuracy_score(y_test_pca, y_pred_pca)
print(f"\nBalanced Accuracy: {balanced_acc_pca:.2f}")

Classification Report:
               precision    recall  f1-score   support

           1       0.37      0.59      0.46       583
           2       0.81      0.68      0.74      3832
           3       0.43      0.55      0.48      1056

    accuracy                           0.65      5471
   macro avg       0.54      0.61      0.56      5471
weighted avg       0.69      0.65      0.66      5471


Balanced Accuracy: 0.61


### **INTERPRETATION:**

- **Balanced Accuracy** and **Accuracy**: 

    - This PCA applied Random Forest Classifier model performs slightly worse than the previous model (61% vs 62% for the balanced accurancy score). 
    - This performs above average, with **65%** of the predictions made by the model are correct. 

- **Precision, Recall, and F1-score (ie. the harmonic mean of precision and recall):** 
    

    - **Precision scores**: 
    
    - Overall the new model performs better than the 2nd model (currently the optimal model) and 1st model. Comparing it to the 2nd model:

            - 3% improvement for Delay Type 1 (37% vs 34%) 
            - 2% worse for Delay Type 2 (81% vs 83%)
            - 1% better for Delay Type 3 (43% vs 42%)

        
        - Good score for the majority class (Delay Type 2 - normal delay) at 81%, meaning there are very few false positive for the normal delay class. 
        - Poor score (many false positives) for the minority classes (Delay Type 1 and 3, at 37% and 43% respectively)


    - **Recall scores**: 
    
        - Overall the new model performs worse than the 2nd model (currently the optimal model) and 1st model:

            - 4% worse for Delay Type 1 (59% vs 63%) 
            - 6% improvement for Delay Type 2 (68% vs 62%)
            - 6% worse for Delay Type 3 (55% vs 61%)
        
        - There are many missed true positive cases, and it performs slightly above average.  

       
    - **F1 scores**: 
        
        - Overall the new model performs better than the 2nd model:

            - 2% improvement for Delay Type 1 (46% vs 44%) 
            - 3% improvement for Delay Type 2 (74% vs 71%)
            - 1% worse for Delay Type 3 (48% vs 49%)
        
        - This means that the model is slightly more effective at correctly identifying positive instances while minimizing false positives for the minority classes compared to the 2nd model. 


### **CONCLUSION:**

- Applying PCA on a reduced dataset overall improves performance across scoring metrics, except for the recall scores where the 2nd model performs better. 
- This 3rd model performs very well at 'precision score' for the majority class (81% for Delay Type 2 - normal delays), meaning there are very few false positives for normal delay type. 
- The new model performs above average for identifying true positive cases for the majority class (Delay Type 2 at 68%) but performs averagely for the minority classes (Delay Types 1 and 2, at 59% and 55% respectively).
- This 3rd model performs acceptably at 'F1 score' for the majority class (74% for Delay Type 2 - normal delay type), meaning it performs acceptably at identifying positive instancees while minimizing false positives for this class. 
- So far this 3rd model is the optimal model.


In [26]:
# Save the trained model to a pkl file in 'data' folder
relative_path = os.path.join("PCA_RF.pkl")
joblib.dump(best_rf_pca, relative_path) 

['PCA_RF.pkl']

---
---